In [ ]:
using System;
static bool debug = false;
public enum Op {none, magnitude, reduce, explode, split, exploded, addright, addleft};
public record Opval
{
    public (int,int) pair;
    public int value;
    public Op op;
    public Opval (Op o, int v) {value = v; op=o;}
    public Opval (Op o, (int,int) p) {pair=p; op=o;}
    public Opval (Op o) {op=o;}
}
public class Snumber
{
    int? value;
    Snumber left;
    Snumber right;
    int level;
    Snumber (Snumber orig, int l=0)
    {
        level = l;
        if (orig.value is not null)
            value=orig.value;
        else
        {
            left = new Snumber(orig.left, l+1);
            right = new Snumber(orig.right, l+1);
        }
    }
    Snumber (int sn, int l=0)
    {
        value = sn;
        level = l;
    }
    public Snumber (string sn, int l=0)
    {
        //Console.WriteLine($"parse {sn} {l}");
        level=l;
        if (sn[0]=='[')
        {
            //node
            left = new Snumber(sn.Substring(1), l+1);
            //find ',' matching level
            var level = 0;
            var pos=0;
            for (var i=1;i<sn.Length;i++)
            {
                if (sn[i]=='[') level++;
                else if (sn[i]==']') level--;
                else if (sn[i]==',' && level==0) {pos=i; break;}
            }
            right = new Snumber(sn.Substring(pos+1), l+1);
        }
        else
        {
            //number
            var pos=sn.IndexOfAny(new char[]{',',']'});
            //Console.WriteLine($"to number {sn.Substring(0,pos)} {pos}");
            value = Int32.Parse(sn.Substring(0,pos));
        }
    }
    public Snumber (Snumber a, Snumber b)
    {
        left = new Snumber(a,1);
        right = new Snumber(b,1);
        level = 0;
        //left.IncreaseLevel();
        //right.IncreaseLevel();
    }
    void IncreaseLevel()
    {
        level++;
        left?.IncreaseLevel();
        right?.IncreaseLevel();
    }
    public Opval Dooperation(Opval ov)
    {
        Opval r;
        switch (ov.op)
        {
            case Op.reduce:
                r=Reduce(new Opval(Op.explode));
                if (r.op==Op.explode) //no explosion
                {
                    r=Reduce(new Opval(Op.split));
                    if (r.op==Op.split) //no split
                        return ov;
                }
                return r;
            default: return new Opval(Op.none);
        }
    }
    Opval Reduce (Opval ov)
    {
        if (debug)
        {
            var f = value??-1;
            Console.WriteLine($"{ov.op.ToString()} ({ov.pair.Item1},{ov.pair.Item2}) {level} {f} {left.value??0},{left.value??0}");
            printsn();
        }
        if (value is not null)
        {
            if (ov.op==Op.split && value>9) //split
            {
                double v = value??0;
                var le = (int)(Math.Floor(v/2));
                var ri = (int)(Math.Ceiling(v/2));
                left = new Snumber(le, level+1);
                right = new Snumber(ri, level+1);
                if (debug)
                {
                    Console.WriteLine($"{value} to {v} to {le},{ri}");
                    left.printsn();
                    right.printsn();
                }
                value = null;
                return new Opval(Op.none);
            }
            return ov;
        }
        if (ov.op==Op.explode && level>=4 && value is null && left.value!=null &&right.value!=null)
        {
            //assume left and right are values
            var re = new Opval(Op.exploded, (left.value??0, right.value??0));
            if (debug)Console.WriteLine($"explosion with {left.value??0},{right.value??0}");
            left=null;
            right=null;
            value=0;
            return re;
        }
        //assume left & right right exists
        var r=left.Reduce(ov);
        if (debug)
            Console.WriteLine($"after {ov.op.ToString()} left {r.op.ToString()}");
        switch(r.op) 
        {
            case Op.addleft:
                return r;
            case Op.addright:
                return right.Addright(r);
            case Op.exploded:
                right.Addright(new Opval(Op.addright, r.pair));
                return new Opval(Op.addleft, r.pair);
            case Op.none: return r;
            default: break;
        }

        r=right.Reduce(ov); //it must be still reduce
        if (debug)
            Console.WriteLine($"after {ov.op.ToString()} right {r.op.ToString()}");
        switch (r.op)
        {
            case Op.addleft:
                return left.Addleft(r);
            case Op.addright:
                return r;
            case Op.exploded:
                left.Addleft(r);
                return new Opval(Op.addright, r.pair);
            case Op.none: return r;
            default: break;
        }
        return ov;
    }
    Opval Addright(Opval ov) 
    {
        if (value is null)
            return left.Addright(ov);
        if (debug) Console.WriteLine($"Addright ({ov.pair.Item1},{ov.pair.Item2}) {value}");
        value+=ov.pair.Item2;
        return new Opval(Op.none);
    }
    Opval Addleft(Opval ov)
    {
        if (value is null)
            return right.Addleft(ov);
        if (debug) Console.WriteLine($"Addleft ({ov.pair.Item1},{ov.pair.Item2}) {value}");
        value+=ov.pair.Item1;
        return new Opval(Op.none);
    }
    public void printsn()
    {
        if (value is not null)
        {
            //Console.Write($"{{{level}}}{value}");
            Console.Write(value);
            return;
        }
        Console.Write($"[");
        left.printsn();
        Console.Write(",");
        right.printsn();
        Console.Write("]");
        if (level==0)
            Console.WriteLine();
    }
    public long CalculateMagnitude()
    {
        if (value is not null)
            return (long) value;
        return 3*left.CalculateMagnitude()+2*right.CalculateMagnitude();
    }
}

In [ ]:
var st = new string[] {
/*"[1,2]",
"[[1,2],3]",
"[9,[8,7]]",
"[[1,9],[8,5]]",
"[[[[1,2],[3,4]],[[5,6],[7,8]]],9]",
"[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]",
"[[[[1,3],[5,3]],[[1,3],[8,7]]],[[[4,9],[6,9]],[[8,2],[7,3]]]]",*/
/*"[[[[[9,8],1],2],3],4]",
"[7,[6,[5,[4,[3,2]]]]]",
"[[6,[5,[4,[3,2]]]],1]",
"[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]",
"[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]",
"[[[[0,7],4],[15,[0,13]]],[1,1]]",*/
/*"[[[[4,3],4],4],[7,[[8,4],9]]]",
"[1,1]",*/
/*"[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]",
"[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]",
"[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]",
"[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]",
"[7,[5,[[3,8],[1,4]]]]",
"[[2,[2,2]],[8,[8,1]]]",
"[2,9]",
"[1,[[[9,3],9],[[9,0],[0,7]]]]",
"[[[5,[7,4]],7],1]",
"[[[[4,2],2],6],[8,7]]",*/
/*"[[1,2],[[3,4],5]]", //becomes 143.
"[[[[0,7],4],[[7,8],[6,0]]],[8,1]]", //becomes 1384.
"[[[[1,1],[2,2]],[3,3]],[4,4]]", //becomes 445.
"[[[[3,0],[5,3]],[4,4]],[5,5]]", //becomes 791.
"[[[[5,0],[7,4]],[5,5]],[6,6]]", //becomes 1137.
"[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]", //becomes 3488.*/
/*"[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]",
"[[[5,[2,8]],4],[5,[[9,9],0]]]",
"[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]",
"[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]",
"[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]",
"[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]",
"[[[[5,4],[7,7]],8],[[8,3],8]]",
"[[9,3],[[9,9],[6,[4,9]]]]",
"[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]",
"[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]",*/
};

/*foreach (var s in st)
{
    var p = new Snumber(s);
    Opval ov ;
    do
    {
        ov = p.Dooperation(new Opval(Op.reduce));
    }
    while (ov.op!=Op.reduce);
    p.printsn();
    Console.WriteLine(p.CalculateMagnitude());
}*/


var a = new Snumber(st.First());
foreach (var s in st.Skip(1))
{
    var p = new Snumber(s);
    //Console.WriteLine($"Add two snumbers");
    //a.printsn();
    //p.printsn();
    var c = new Snumber(a,p);
    c.printsn();
    Opval ov ;
    do
    {
        ov = c.Dooperation(new Opval(Op.reduce));
        //c.printsn();
    }
    while (false && ov.op!=Op.reduce);
    a=c;
}
a.printsn();
Console.WriteLine(a.CalculateMagnitude());
Console.WriteLine($"done");   


[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
explode (0,0) 0 -1 0,0
[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
explode (0,0) 1 -1 0,0
[[[[4,3],4],4],[7,[[8,4],9]]]explode (0,0) 2 -1 0,0
[[[4,3],4],4]explode (0,0) 3 -1 0,0
[[4,3],4]explode (0,0) 4 -1 4,4
[4,3]explosion with 4,3
after explode left exploded
Addright (4,3) 4
after explode left addleft
after explode left addleft
after explode left addleft
[[[[0,7],4],[7,[[8,4],9]]],[1,1]]
1954
done


In [ ]:
using System.IO;
var filedata = File.ReadAllLines(@"..\day18.input");
var a = new Snumber(filedata.First());
foreach (var s in filedata.Skip(1))
{
    var p = new Snumber(s);
    /*Console.WriteLine($"Add two snumbers");
    a.printsn();
    p.printsn();*/
    var c = new Snumber(a,p);
    //c.printsn();
    Opval ov ;
    do
    {
        ov = c.Dooperation(new Opval(Op.reduce));
    }
    while (ov.op!=Op.reduce);
    a=c;
}
a.printsn();
Console.WriteLine(a.CalculateMagnitude());
Console.WriteLine($"done");   
//4140 too high?


[[[[6,6],[7,0]],[[7,7],[6,6]]],[[[5,6],[0,8]],[[0,6],[9,0]]]]
3216
done


In [ ]:
var test = new string[] {
"[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]",
/*"[[[5,[2,8]],4],[5,[[9,9],0]]]",
"[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]",
"[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]",
"[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]",
"[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]",
"[[[[5,4],[7,7]],8],[[8,3],8]]",
"[[9,3],[[9,9],[6,[4,9]]]]",*/
"[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]",
//"[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]",
};
var nl = filedata.Select(x=>new Snumber(x));
//var nl = test.Select(x=>new Snumber(x)).ToList();
long max = 0;
int rep =0;
//for (var i=0;i<nl.Count;i++)
foreach ((Snumber it,int i) in nl.Select((x,i)=>(x,i)))
{
    //for (var j=0;j<nl.Count;j++)
    foreach ((Snumber it2,int j) in nl.Select((x,i)=>(x,i)))
    {
        if (j==i) continue;
        rep++;
        //var c = new Snumber(nl[i],nl[j]);
        var c = new Snumber(it,it2);
        if (debug)
        {
            Console.WriteLine("add 2");
            it.printsn();
            it2.printsn();
            c.printsn();
        }
        Opval ov ;
        do
        {
            ov = c.Dooperation(new Opval(Op.reduce));
            //c.printsn();
        }
        while (ov.op!=Op.reduce);
        var m=c.CalculateMagnitude();
        //c.printsn();
        if (debug) Console.WriteLine($"{m} {max}");
        if (m>max)
            max=m;
    }
}
Console.WriteLine(max);
Console.WriteLine($"{rep} done");   
//2956 too low


4643
9900 done
